# StackAI Vector Database Exploration

Interactive notebook for testing the StackAI vector database API.

**Setup (Local Development):**
1. Start the server: `make start`
2. Seed test data: `python scripts/seed_data.py --library all`
3. Run the cells below

**Setup (Docker):**
1. Start the container: `docker compose up --build`
2. Seed test data: `docker compose exec api python scripts/seed_data.py --library all`
3. Run the cells below

The API is available at `http://localhost:8000` in both cases.

In [ ]:
import sys
sys.path.insert(0, "..")

from client import StackAIClient

# Create client instance
client = StackAIClient()

# Convenience aliases for common operations
search = client.print_search
list_libraries = client.print_libraries
list_documents = client.print_documents
list_chunks = client.print_chunks
health_check = client.print_health

print("Client loaded. Available functions: search(), list_libraries(), list_documents(), list_chunks(), health_check()")
print("Direct API access via: client.create_library(), client.search(), etc.")

## Quick Start

In [2]:
health_check()

Server is running


In [3]:
list_libraries()

Libraries (3):
  - recipes_lib: Recipe Collection
  - support_lib: Support Knowledge Base
  - products_lib: Product Manuals


## Search

Available libraries (after seeding):
- `recipes_lib` - Cooking recipes
- `support_lib` - Support knowledge base
- `products_lib` - Product manuals

In [ ]:
search("How do I make a creamy pasta sauce?", library_id="recipes_lib")

In [ ]:
search("How do I reset my password?", library_id="support_lib")

In [ ]:
search("bluetooth pairing", library_id="products_lib", k=5)

In [ ]:
# Try your own query
search("chicken curry recipe", library_id="recipes_lib", k=5)

## Browse Data

In [8]:
list_documents("recipes_lib")

Documents in recipes_lib (5):
  - recipes_spaghetti_carbonara: Spaghetti Carbonara
  - recipes_thai_green_curry: Thai Green Curry
  - recipes_chocolate_chip_cookies: Chocolate Chip Cookies
  - recipes_chicken_tikka_masala: Chicken Tikka Masala
  - recipes_french_onion_soup: French Onion Soup


In [9]:
list_chunks("recipes_spaghetti_carbonara")

Chunks in recipes_spaghetti_carbonara (10):
  [recipes_spaghetti_carbonara_chunk_1] Spaghetti Carbonara is a classic Roman pasta dish made with eggs, cheese, and cu...
  [recipes_spaghetti_carbonara_chunk_2] You'll need 400g spaghetti, 200g guanciale, 4 egg yolks, 100g pecorino romano, a...
  [recipes_spaghetti_carbonara_chunk_3] Bring a large pot of salted water to boil and cook the spaghetti until al dente.
  [recipes_spaghetti_carbonara_chunk_4] Cut the guanciale into small strips and crisp it in a dry pan over medium heat.
  [recipes_spaghetti_carbonara_chunk_5] Whisk together the egg yolks, grated pecorino, and plenty of black pepper in a b...
  [recipes_spaghetti_carbonara_chunk_6] Reserve a cup of pasta water before draining the cooked spaghetti.
  [recipes_spaghetti_carbonara_chunk_7] Toss the hot pasta with the guanciale and rendered fat, then remove from heat.
  [recipes_spaghetti_carbonara_chunk_8] Stir in the egg mixture quickly, adding pasta water as needed to create a cre

## Manual API Calls

For more control, use the client directly:

In [ ]:
# Raw API call example - using client methods
result = client.search("recipes_lib", "baking cookies", k=2)
result

## Testing Deletes

Test that deleting chunks/documents/libraries properly removes them from both storage and the search index.

In [ ]:
# Create a test library for delete testing
client.create_library("delete_test_lib", "Delete Test Library")
print("Created library")

# Create a document
client.create_document("delete_test_lib", "delete_test_doc", "Test Document")
print("Created document")

# Create chunks
chunks = [
    {"id": "chunk_a", "document_id": "delete_test_doc", "text": "The quick brown fox jumps over the lazy dog"},
    {"id": "chunk_b", "document_id": "delete_test_doc", "text": "Machine learning is a subset of artificial intelligence"},
    {"id": "chunk_c", "document_id": "delete_test_doc", "text": "Python is a popular programming language"},
]
result = client.create_chunks_batch("delete_test_doc", chunks)
print(f"Created {result.get('created_count', 0)} chunks")

In [ ]:
# Verify search works - search for "programming"
print("=== BEFORE DELETE ===")
search("programming language", library_id="delete_test_lib", k=3)

In [ ]:
# Delete single chunk (chunk_c - the Python one)
client.delete_chunk("chunk_c")
print("Deleted chunk_c")

# Verify it's gone from search results
print("\n=== AFTER DELETING chunk_c ===")
search("programming language", library_id="delete_test_lib", k=3)
# Should NOT find the Python chunk anymore

In [ ]:
# Create another document with chunks to test document deletion
client.create_document("delete_test_lib", "delete_test_doc2", "Second Test Document")
print("Created document2")

chunks2 = [
    {"id": "chunk_d", "document_id": "delete_test_doc2", "text": "Cats are popular pets around the world"},
    {"id": "chunk_e", "document_id": "delete_test_doc2", "text": "Dogs are known as man's best friend"},
]
client.create_chunks_batch("delete_test_doc2", chunks2)
print("Created chunks")

# Verify we can find pet content
print("\n=== BEFORE DOCUMENT DELETE ===")
search("pets and animals", library_id="delete_test_lib", k=5)

In [ ]:
# Delete the document (should cascade delete its chunks from index too)
client.delete_document("delete_test_lib", "delete_test_doc2")
print("Deleted document2")

# Verify pet chunks are gone from search
print("\n=== AFTER DOCUMENT DELETE ===")
search("pets and animals", library_id="delete_test_lib", k=5)
# Should NOT find cats or dogs chunks anymore

In [ ]:
# Test library deletion (deletes everything including index file)
client.delete_library("delete_test_lib")
print("Deleted library")

# Verify library is gone
import httpx
try:
    client.get_library("delete_test_lib")
    print("ERROR: Library still exists!")
except httpx.HTTPStatusError as e:
    print(f"Get deleted library: {e.response.status_code} (expected 404)")

# Verify search fails gracefully
try:
    client.search("delete_test_lib", "test", k=3)
    print("ERROR: Search should have failed!")
except httpx.HTTPStatusError as e:
    print(f"Search deleted library: {e.response.status_code} (expected 404)")

print("\nDelete tests complete!")

## Testing Index Persistence

Test that indexes survive server restarts.

**Manual test:**
1. Run the cell below to create test data
2. Restart the server (Ctrl+C, then `make start`)
3. Run the verification cell to confirm search still works

In [ ]:
# Step 1: Create test data for persistence test
client.create_library("persist_test_lib", "Persistence Test Library")
print("Created library")

client.create_document("persist_test_lib", "persist_test_doc", "Persistence Test Doc")
print("Created document")

persist_chunks = [
    {"id": "persist_chunk_1", "document_id": "persist_test_doc", "text": "This chunk should survive a server restart"},
    {"id": "persist_chunk_2", "document_id": "persist_test_doc", "text": "Index persistence means the search index is saved to disk"},
]
client.create_chunks_batch("persist_test_doc", persist_chunks)
print("Created chunks")

print("\nTest data created. Now restart the server and run the next cell.")

In [ ]:
# Step 2: Verify search still works after restart
# (Run this AFTER restarting the server)

print("Checking if search works after restart...")
search("server restart persistence", library_id="persist_test_lib", k=3)

# If results appear, persistence is working!
print("\nIf you see results above, index persistence is working!")

In [ ]:
# Cleanup: Delete persistence test library
client.delete_library("persist_test_lib")
print("Test data cleaned up")